# PageRank

[ref1: PageRank算法原理及实现](https://blog.csdn.net/rubinorth/article/details/52215036)  
[ref2: Topic PageRank原理](https://blog.csdn.net/hguisu/article/details/8005192)

## python-graph-core

In [ ]:
!pip install python-graph-core

In [2]:
from pygraph.classes.digraph import digraph

In [3]:
dg = digraph()

dg.add_nodes(["A", "B", "C", "D"])

dg.add_edge(("A", "B"))
dg.add_edge(("A", "C"))
dg.add_edge(("A", "D"))
dg.add_edge(("B", "A"))
dg.add_edge(("B", "D"))
dg.add_edge(("C", "C"))

In [4]:
dg.nodes()[1]

'B'

## Page Rank

In [5]:
class PageRank:
    
    def __init__(self, dg):
        self.digraph = dg
        self.nodes_num = len(dg.nodes())
        self.damping_factor = 0.85
        self.remain_part = (1 - self.damping_factor) / self.nodes_num
        self.threshold = 0.000000000001
            
        # supply edges
        for node in self.digraph:
            if(len(self.digraph.neighbors(node)) == 0):
                for node2 in self.digraph:
                    self.digraph.add_edge((node, node2))
    
    
    def page_rank(self):
        # init
        pr = dict.fromkeys(self.digraph.nodes(), 1.0 / self.nodes_num) 
        
        # iteration
        diff = self.threshold
        round = 0
        while(diff >= self.threshold):
            diff = 0
            round += 1
            pr_copy = pr.copy()    #set copy to make sure each iteration, the sum of pr value is 1
            for node in self.digraph:
                cur_points = 0
                for in_node in self.digraph.incidents(node):
                    cur_points += self.damping_factor * (pr[in_node] / len(self.digraph.neighbors(in_node)))
                cur_points += self.remain_part
                diff += abs(pr[node] - cur_points)
                pr_copy[node] = cur_points
                
                
            pr = pr_copy
#             total = 0
#             for i in pr.values():
#                 total += i         
#             print(round, ':', pr, '---------', total)
        
        return pr

In [6]:
PR = PageRank(dg)
pr = PR.page_rank()
pr

{'A': 0.10883978486662309,
 'B': 0.09801945537695188,
 'C': 0.6534630358442265,
 'D': 0.13967772391219846}

## Topic page rank

In [7]:
class TopicPageRank:
    
    def __init__(self, dg, topic_matrix):
        self.digraph = dg
        self.nodes_num = len(dg.nodes())
        self.damping_factor = 0.85
        self.remain_part = (1 - self.damping_factor)     # need topic part
        self.threshold = 0.000000000001
            
        # supply edges
        for node in self.digraph:
            if(len(self.digraph.neighbors(node)) == 0):
                for node2 in self.digraph:
                    self.digraph.add_edge((node, node2))
    
    
    def topic_page_rank(self):
        # init
        pr = dict.fromkeys(self.digraph.nodes(), 1.0 / self.nodes_num) 
        
        pr_list = []
        # iteration
        for cur_topic in topic_matrix:
            diff = self.threshold
            round = 0
            while(diff >= self.threshold):
                diff = 0
                round += 1
                pr_copy = pr.copy()    #set copy to make sure each iteration, the sum of pr value is 1
                for i in range(len(self.digraph)):
                    node = self.digraph.nodes()[i]
                    cur_points = 0
                    for in_node in self.digraph.incidents(node):
                        cur_points += self.damping_factor * (pr[in_node] / len(self.digraph.neighbors(in_node)))
                    cur_points += self.remain_part * cur_topic[i] / sum(cur_topic)
                    diff += abs(pr[node] - cur_points)
                    pr_copy[node] = cur_points


                pr = pr_copy
                
            pr_list.append(list(pr.values()))
                
#                 total = 0
#                 for i in pr.values():
#                     total += i
#                 print(round, ':', pr, '---------', total)
        
        return pr_list

In [8]:
#topic matrix's size is web_nums*topic_nums
topic_matrix = [[0, 1, 1, 0], [1, 0, 0, 0]]

Topic_PR = TopicPageRank(dg, topic_matrix)
topic_pr = Topic_PR.topic_page_rank()
topic_pr

[[0.06492197693812396,
  0.11109932426005663,
  0.7406621617311886,
  0.08331653707063072],
 [0.21300136845316933,
  0.08656263591689722,
  0.5770842394484029,
  0.1233517561815305]]

In [9]:
user_prefer = [0.4, 0.6]

final_rank = []
for i in range(len(topic_pr[0])):
    web_rank = 0
    for j in range(len(user_prefer)):
        web_rank += topic_pr[j][i] * user_prefer[j]
    final_rank.append(web_rank)

final_rank

[0.15376961184715118,
 0.09637731125416099,
 0.6425154083615172,
 0.1073376685371706]